# Libraries

In [16]:
from sknn.mlp import Classifier, Layer, Convolution
import tensorflow as tf
import numpy as np
from numpy import *
from sklearn import preprocessing as prep
import sklearn.cross_validation as crval

import pymc3 as pm
import theano.tensor as T
import theano
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_moons

# Functions

In [48]:
# GENERAL
def test_and_answers(solver, test):
    print solver
    print shape(test)
    accuracy = solver.predict(test)
    print 'Saving answers...'
    fp=open('answers.txt','w')
    fp.write('Point_ID,Output\n')
    i=1
    for a in accuracy:
        #fp.write(str(i)+','+a+'\n')
        fp.write(str(i)+','+a)
        i+=1
    fp.close()
    print 'Done.'
    
    

def compute_accuracy_val(target, y):
    cont=0
    for i in range(len(y)):
        if y[i] is (target[i]):
            cont+=1
    return round(float(cont*(pow(len(target),-1))),4)
    

In [3]:
# TENSORFLOW

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    #out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer



# Preprocessing

In [4]:
#LOADING DATA ..... 
    
X=[]
fp=open('class_train_in.csv','r')
trash=fp.readline()
#print trash
lines=fp.readlines()
# print lines[0]
for line in lines:
    data=line.split(',')
    cleaned_data=data[1:len(data)]
    X.append(cleaned_data)
print shape(X)

      
y=[]
fp=open('class_train_out.csv','r')
trash=fp.readline()
#print trash
lines=fp.readlines()
# print lines[0]
for line in lines:
    data=line.split(',')
    cleaned_data=data[1:len(data)]
    y.append(cleaned_data)
print shape(y)
    
    
X_test=[]
fp=open('class_test_in.csv','r')
trash=fp.readline()
lines=fp.readlines()
for line in lines:
    data=line.split(',')
    cleaned_data=data[1:len(data)]
    X_test.append(cleaned_data)
print shape(X_test)

(1962, 265)
(1962, 1)
(1963, 265)


In [5]:
# SET UP DATA ARRAYS
X=np.asarray(X)
X=X.astype(np.float)
X=np.ravel(X)
X= X.reshape(1962, 265)
print shape(X)

y=np.asarray(y)
y=np.ravel(y).reshape(1962, 1)
print shape(y)

X_test=np.asarray(X_test)
X_test=X_test.astype(np.float)
X_test=np.ravel(X_test)
X_test= X_test.reshape(1963, 265)
print shape(X_test)


(1962, 265)
(1962, 1)
(1963, 265)


In [6]:
#NORMALIZING DATA
print 'X train:'
std_scal=prep.StandardScaler()
X_scaled=std_scal.fit_transform(X)
print('it had mean: ' +str(mean(X))+' and std:'+ str(std(X)))
print('NOW it has mean: '+str(mean(X_scaled))+' and std: '+str(std(X_scaled)))

X_tr = X_scaled #normalized training data
y_tr = y


print 'X test:'
selX=X_test
std_scal=prep.StandardScaler()
selX_scaled=std_scal.fit_transform(selX)
print('it had mean: ' +str(mean(selX))+' and std:'+ str(std(selX)))
print('NOW it has mean: '+str(mean(selX_scaled))+' and std: '+str(std(selX_scaled)))

X_test = selX_scaled #normalized testing data

X train:
it had mean: 93.6561498663 and std:24634.8063775
NOW it has mean: -7.3728733857e-18 and std: 1.0
X test:
it had mean: 93.7580887936 and std:24628.007464
NOW it has mean: -1.45060291886e-17 and std: 1.0


In [7]:
# SHUFFLE AND SPLIT
print('SHUFFLE: Lets mess up things a little bit')
selX_scaled=X
sely=y
indexes=[]
shuffly=y
shufflX=X
# assign an id to each sample
for i in range(len(sely)):
    indexes.append(i)
# shuffle idxs
np.random.shuffle(indexes)
# build X and y shuffled
for i in range(len(indexes)):
    shuffly[i]=sely[i]
    shufflX[i, :]= selX_scaled[i,:]
print('total matrix of data:'+str(shape(shufflX)))


print('SPLIT: create of train/test/val sets')
trainData, valData, trainLab, valLab = crval.train_test_split(shufflX, shuffly, test_size=0.2)

#trainData, valData, trainLab, valLab = crval.train_test_split(shufflX, shuffly, test_size=0.2)
#print(shape(otherData))
print('Done. Here the sizes: ')
print('Train: '+str(shape(trainData))+';\tVal: '+str(shape(valData)))

SHUFFLE: Lets mess up things a little bit
total matrix of data:(1962, 265)
SPLIT: create of train/test/val sets
Done. Here the sizes: 
Train: (1569, 265);	Val: (393, 265)


# MLP

In [ ]:
# NEURAL NET, TRY
#from scikit.neuralnetwork import MLPClassifier
from sknn.mlp import Classifier, Layer, Convolution

#class sknn.mlp.Classifier(layers, warning=None, parameters=None, random_state=None, learning_rule=u'sgd', learning_rate=0.01, learning_momentum=0.9, normalize=None, regularize=None, weight_decay=None, dropout_rate=None, batch_size=1, n_iter=None, n_stable=10, f_stable=0.001, valid_set=None, valid_size=0.0, loss_type=None, callback=None, debug=False, verbose=None, **params)

##mlp = Classifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, learning_rate='constant', learning_rate_init=0.001)
#mlp = Classifier(solver='lbfgs', alpha=1e-5, random_state=1, learning_rule='constant', learning_rate=0.001)
#mlp.fit(trainData, trainLab)

#MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',beta_1=0.9, beta_2=0.999, early_stopping=False,
    #epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant', learning_rate_init=0.001, max_iter=200,
    #momentum=0.9, nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
    #validation_fraction=0.1, verbose=False, warm_start=False)

typeslin = ['Rectifier', 'Sigmoid', 'Tanh']
typesnonlin = ['ExpLin']
typesclass = ['Linear', 'Softmax']
    
layers=[
        Layer("Sigmoid", units=100),
        #Layer("Sigmoid", units=50),
        Layer("Rectifier", units=100),
        #Layer('ExpLin', units=100),
        Layer("Softmax")]

#layers=[Layer('Sigmoid', units=100) for i in range(5)]
print layers
    
#mlp = Classifier(layers, learning_rate=0.1, n_iter=50, regularize='dropout', random_state=1)
mlp = Classifier(layers, learning_rate=0.1, n_iter=50, regularize='dropout', random_state=1,learning_rule='adadelta')
mlp.fit(trainData, trainLab)
    
print 'Trained'
# to be continued...

In [ ]:
# ...continuing (NN)
# on validation
y = mlp.predict(valData)
cont=0
for i in range(len(y)):
    if y[i]==valLab[i]:
        cont+=1
accuracy = round(float(cont*(pow(len(valLab),-1))),4)
print accuracy

# [sig(100), rect(100)], lr=0.1, niter=50, regul=dropout, rndstate=1 -> acc=0.7968 = 4answers.csv
# [sig(100), rect(100)], lr=0.1, niter=50, regul=dropout, rndstate=1 -> acc=



In [ ]:
# ...continuing (NN)
# on test
y = mlp.predict(X_test)
accuracy = y

fp=open('answers.txt','w')
fp.write('Point_ID,Output\n')
i=1
for a in accuracy:
    fp.write(str(i)+','+str(a[0])+'\n')
    i+=1
fp.close()

# Tensorflow

In [ ]:
# DEFINE VARIABLES AND PARAM

n_feat = len(trainData[1,:])
n_classes = 1
n_samples = len(trainLab)

# Y = W*X+b
x = tf.placeholder(tf.float32, [None, n_feat])
y = tf.placeholder(tf.float32, [None, n_classes])
# initialize weights and bias
#W = tf.Variable(tf.zeros([n_feat, 2]))
#b = tf.Variable(tf.zeros([2]))
W = tf.Variable(tf.random_normal([n_feat, 2]))
b = tf.Variable(tf.random_normal([2]))

#trainD = tf.constant(trainData)
#trainD = trainData.as_matrix()
#trainL = tf.constant(trainLab)
#print trainD.get_shape()

# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 100
display_step = 10


output = tf.nn.softmax(tf.matmul(x, W) + b)


print trainData
#with tf.Session().as_default():
#    print x.eval()




In [ ]:
# METHOD 1

# define loss function to minimize:
#loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(output), reduction_indices=[1])) #cross entropy
loss = tf.reduce_sum(tf.pow(y - output, 2))/(2*n_samples) # mean squared error

# define training step:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# initialize net:
init = tf.global_variables_initializer()

# initialize training session:
with tf.Session() as sess:
    sess.run(init)

    # training
    print 'Training...'
    for i in range(training_epochs):
        #batch_xs, batch_ys = trainD.next_batch(batch_size), trainL.next_batch(batch_size)
        sess.run(train_step, feed_dict={x: trainData, y: trainLab})
        output = tf.nn.softmax(tf.matmul(x, W) + b)
        print output.eval(session=sess)
        loss = tf.reduce_sum(tf.pow(y - output, 2))/(2*n_samples) # mean squared error
        #print x
        if not (i) % display_step:
            c = sess.run(loss, feed_dict={x:trainData, y: trainLab})
            print "i = {}; \tloss = {}".format(i,c)


print 'Done.'

In [ ]:
# METHOD 2

# Store layers weight & bias
n_hidden_1 = 256
n_hidden_2 = 256

weights = {
    'h1': tf.Variable(tf.random_normal([n_feat, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
loss = tf.reduce_mean()
#loss = tf.reduce_mean(tf.nn.softmax(pred))
#train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(len(trainLab)/batch_size)
        s = 0
        # Loop over all batches
        for i in range(total_batch):
            #batch_x, batch_y = trainData[s:s+batch_size], [int(trainLab[j][0].strip()) for j in range(s,batch_size)]
            batch_x = trainData[s:s+batch_size]
            
            batch_y = np.array([int(trainLab[j][0]) for j in range(s,batch_size)]).ravel().reshape(batch_size,1)
            #batch_x, batch_y = trainData[s:s+batch_size], trainLab[s:s+batch_size]
            print shape(batch_y)
            #batch_x, batch_y = trainData, trainLab
            # Run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_loss += float(c / total_batch)
            print c
            s+=batch_size
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "loss =", "{:.9f}".format(c)
            #print "Epoch:", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_loss)
            #print "epoch = {}; \tloss = {.6f}".format(epoch,c)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #a = sess.run([accuracy, correct_prediction], feed_dict={x: trainData, y: trainLab})
    print "Accuracy:", accuracy.eval({x: valData, y: valLab})
    #print "Accuracy:", a


In [ ]:
   
# performance evaluation
correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# accuracy on validation
print(sess.run(accuracy, feed_dict={x: valData, y: valLab}))

# Bayesian NN

In [18]:
# Trick: Turn inputs and outputs into shared variables. 
# It's still the same thing, but we can later change the values of the shared variable 
# (to switch in the test-data later) and pymc3 will just use the new data. 
# Kind-of like a pointer we can redirect.
# For more info, see: http://deeplearning.net/software/theano/library/compile/shared.html

n_feat = len(trainData[1,:])
n_classes = 1
n_samples = len(trainLab)


ann_input = theano.shared(trainData)
ann_output = theano.shared(trainLab)

n_hidden = 5

# Initialize random weights between each layer
init_1 = np.random.randn(n_feat, n_hidden)
init_2 = np.random.randn(n_hidden, n_hidden)
init_out = np.random.randn(n_hidden)
#init_1 = np.random.randn(X.shape[1], n_hidden)
#init_2 = np.random.randn(n_hidden, n_hidden)
#init_out = np.random.randn(n_hidden)
    
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    weights_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                             shape=(n_feat, n_hidden), 
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    weights_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                            shape=(n_hidden, n_hidden), 
                            testval=init_2)
    
    # Weights from hidden layer to output
    weights_2_out = pm.Normal('w_2_out', 0, sd=1, 
                              shape=(n_hidden,), 
                              testval=init_out)
    
    # Build neural-network using tanh activation function
    act_1 = T.tanh(T.dot(ann_input, 
                         weights_in_1))
    act_2 = T.tanh(T.dot(act_1, 
                         weights_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, 
                                   weights_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', 
                       act_out,
                       observed=ann_output.eval())

[[ 33884.  30110.  -2466. ..., -25072. -13751. -16103.]
 [ 34867.  28911.   -377. ..., -27951. -13897. -11782.]
 [ 35224.  30093.  -3251. ..., -25687. -15908. -12349.]
 ..., 
 [ 33560.  33436.  -3373. ..., -23780. -11747. -16528.]
 [ 35432.  29254.   -749. ..., -25775. -13450. -13224.]
 [ 33047.  30679.  -1288. ..., -26765. -11867. -13894.]]


In [22]:
with neural_network:
    # Run ADVI which returns posterior means, standard deviations, and the evidence lower bound (ELBO)
    v_params = pm.variational.advi(n=50000)

Average ELBO = -1.7065e+06: 100%|██████████| 50000/50000 [47:28<00:00, 18.74it/s]
Finished [100%]: Average ELBO = -1.7064e+06


In [23]:
with neural_network:
    trace = pm.variational.sample_vp(v_params, draws=5000)

100%|██████████| 5000/5000 [00:00<00:00, 5340.48it/s]


In [49]:
# Replace shared variables with testing set
ann_input.set_value(valData)
ann_output.set_value(valLab)

# Creater posterior predictive samples
ppc = pm.sample_ppc(trace, model=neural_network, samples=500)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['out'].mean(axis=0) > 0.5

#print('Accuracy = {}%'.format((valLab == pred).mean() * 100))
print [int(valLab[j][0].strip()) for j in range(len(valLab))]
print list(pred*1)
print('Accuracy = {}%'.format(compute_accuracy_val(valLab, list(pred*1))))

100%|██████████| 500/500 [00:03<00:00, 130.74it/s]


1

[1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 